# Language Translator 

This is a multi-functional language translator built with Streamlit. It offers `text translation`, `speech-to-text`, `text-to-speech conversion`, and `translation history tracking`.

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://languageicon.org/language-icon.png"> 
</p>

</div>

## Description

- `Text Translation:`
    - Automatically translates text between multiple languages.
    - Supports source language auto-detection.
    - Supports a wide range of target languages.
    - Displays translated text.

- `Speech-to-Text:`
    - Converts uploaded audio files (WAV format) to text.
    - Uses Google Speech Recognition for accurate text conversion.

- `Text-to-Speech:`
    - Converts text input into speech.
    - Allows selection of a language for speech output.
    - Provides a download link for the generated speech as an MP3 file.

- `Translation History:`

    - Keeps track of all translation activities.
    - Displays previous source text and its translated counterpart.
    - Option to clear the history.

Simple and user-friendly design built with `Streamlit`.


## Steps to Use the Language Translator App:
- Launch the application by running streamlit run your_app_name.py.
- The app interface will open in your browser with the following features:


## Step 1: Install Requirements

To run this application, ensure you have Python installed on your system. Then, follow these steps:

1. Create a virtual environment (optional but recommended):

- Install all the requirements that are specified in the `requirements.txt` file in the root folder. 
- Make sure you set up a virtual environment before installing requirements. 
- To setup a virtual environment, run `python3 -m venv env`. 
- To activate the env on Linux / MacOS, run `. env/bin/activate` (for Windows the command is different, figure it out).

In [ ]:
import os
os.system('pip install -r requirements.txt')

## Step 2: Import Libraries

These imports bring in the necessary libraries for the application:

- `streamlit` for creating the web interface.

- `googletrans` for translating text.

- `pyttsx3` for converting text to speech.

- `os` and `base64` for file handling and encoding.

- `speech_recognition` for converting speech (audio) to text.

In [ ]:
import streamlit as st
from googletrans import Translator, LANGUAGES
import pyttsx3
import os
import base64
import speech_recognition as sr


## Step 3: Page Configuration

This sets up the Streamlit page configuration, including:

- The page title.
- The layout (wide to make use of the space).
- A page icon to represent the app.




In [ ]:
st.set_page_config(
    page_title="Professional Language Translator", layout="wide", page_icon="🌐"
)


## Step 4: Add a Title and Description

This sets the main title of the app as `Translator App` and adds a markdown text below the title that describes the app as a language translator app. This helps users understand what the app does and how it works.

In [ ]:
st.title("🌐 Professional Language Translator")
st.markdown("Translate text, detect languages, and convert between text and speech with ease.")


## Step 5: Initialize Translator

This initializes the Google Translate API, enabling translation capabilities for the app.




In [ ]:
translator = Translator()


## Step 6: Sidebar for Feature Selection

This creates a sidebar where the user can select one of four features:

- Text Translation

- Speech-to-Text

- Text-to-Speech

- Translation History


In [ ]:
st.sidebar.title("Features")
feature = st.sidebar.radio(
    "Choose a feature",
    ["Text Translation", "Speech-to-Text", "Text-to-Speech", "Translation History"]
)


## Step 7:  Initialize Session State

This ensures that the translation history is stored in the `session state`, so it persists across user interactions. If no history exists, it initializes an empty list.

In [ ]:
if "history" not in st.session_state:
    st.session_state.history = []


## Step 8: Function to Get Language Code

This helper function takes the language name as input and returns the corresponding language code required by the Google Translate API.

In [ ]:
def get_language_code(language):
    return list(LANGUAGES.keys())[list(LANGUAGES.values()).index(language.lower())]


# Feature: Text Translation

## Step 9: Text Translation Input

This part shows the text input area for users to enter the text they want to translate. It also provides drop-down menus to select the source and target languages.

In [ ]:
if feature == "Text Translation":
    st.header("Text Translation")
    text_to_translate = st.text_area("Enter text to translate:", placeholder="Type something here...")
    source_language = st.selectbox(
        "Select source language:",
        ["Auto-detect"] + [lang.capitalize() for lang in LANGUAGES.values()]
    )
    target_language = st.selectbox(
        "Select target language:",
        [lang.capitalize() for lang in LANGUAGES.values()]
    )


## Step 10: Translation Logic and Display

- When the "Translate" button is pressed, the entered text is sent for translation.

- If the source language is "Auto-detect," the system automatically detects it; otherwise, it uses the selected language.

- The translated text is displayed and added to the translation history.

In [ ]:
if st.button("Translate"):
    if text_to_translate.strip():
        try:
            src_code = (
                "auto"
                if source_language == "Auto-detect"
                else get_language_code(source_language)
            )
            dest_code = get_language_code(target_language)
            
            translation = translator.translate(text_to_translate, src=src_code, dest=dest_code)
            st.success(f"**Translated Text ({target_language}):**")
            st.write(translation.text)
            
            st.session_state.history.append({
                "source_text": text_to_translate,
                "translated_text": translation.text,
                "source_lang": source_language,
                "target_lang": target_language
            })
        except Exception as e:
            st.error(f"Error: {e}")
    else:
        st.warning("Please enter text to translate!")


# Feature: Speech-to-Text

## Step 11: Upload Audio File for Speech-to-Text

- The user uploads a .wav audio file, which is then processed by the Speech Recognition library.

- The audio file is converted to text, which is then displayed.


In [ ]:
elif feature == "Speech-to-Text":
    st.header("Speech-to-Text")
    st.markdown("Upload an audio file, and we will convert it to text.")
    
    uploaded_file = st.file_uploader("Upload audio file (WAV format):", type=["wav"])
    
    if uploaded_file is not None:
        recognizer = sr.Recognizer()
        with sr.AudioFile(uploaded_file) as source:
            audio = recognizer.record(source)
        
        try:
            text = recognizer.recognize_google(audio)
            st.success("Converted Text:")
            st.write(text)
        except Exception as e:
            st.error(f"Error in speech recognition: {e}")

# Feature: Text-to-Speech


## Step 12: Text-to-Speech Input and Logic

- The user enters text to be converted into speech.

- The text is processed using the pyttsx3 engine, which converts the text to an audio file.

- The audio file is played directly in the app and also provided as a download link.

- The temporary audio file is deleted after playback.

In [ ]:
elif feature == "Text-to-Speech":
    st.header("Text-to-Speech")
    
    text_to_convert = st.text_area("Enter text to convert to speech:")
    language = st.selectbox("Select language:", [lang.capitalize() for lang in LANGUAGES.values()])
    
    if st.button("Convert to Speech"):
        if text_to_convert.strip():
            try:
                engine = pyttsx3.init()
                rate = engine.getProperty('rate')
                engine.setProperty('rate', rate )
                
                audio_file = "output.mp3"
                engine.save_to_file(text_to_convert, audio_file)
                engine.runAndWait()
                with open(audio_file, "rb") as file:
                    audio_bytes = file.read()
                    b64 = base64.b64encode(audio_bytes).decode()
                    href = f'<a href="data:audio/mp3;base64,{b64}" download="translated_audio.mp3">Download Audio</a>'
                    st.markdown(href, unsafe_allow_html=True)
                    st.audio(audio_bytes, format="audio/mp3")
                
                os.remove(audio_file)
            except Exception as e:
                st.error(f"Error in text-to-speech conversion: {e}")
        else:
            st.warning("Please enter text to convert!")

# Feature: Translation History

## Step 13: View and Clear Translation History

- Displays the translation history from the session state. 

- If translations exist, they are shown with their source and target languages.

- Users can clear the history.



In [ ]:
elif feature == "Translation History":
    st.header("Translation History")
    
    if st.session_state.history:
        for i, entry in enumerate(st.session_state.history):
            st.write(f"**{i+1}. Source ({entry['source_lang']}):** {entry['source_text']}")
            st.write(f"**Translated ({entry['target_lang']}):** {entry['translated_text']}")
            st.markdown("---")
        
        if st.button("Clear History"):
            st.session_state.history = []
            st.success("Translation history cleared!")
    else:
        st.info("No translations yet!")


## Footer

In [ ]:
st.markdown(
        """
        <style>
        .bottom-right {
            position: fixed;
            bottom: 10px;
            right: 15px;
            font-size: 0.9em;
            color: gray;
        }
        </style>
        <div class="bottom-right">
            Made with ⚡ at 'The Hackers Playbook' ©. All rights reserved.
        </div>
        """,
        unsafe_allow_html=True
                )


## Step 14: Launch Application:
Run the application by executing `streamlit run lang_translator.py`.


In [ ]:
streamlit run lang_translator.py

## Here is the complete code for the Translator App

```python

import streamlit as st
from googletrans import Translator, LANGUAGES
import pyttsx3
import os
import base64
import speech_recognition as sr

# Page Configuration
st.set_page_config(
    page_title="Professional Language Translator", layout="wide", page_icon="🌐"
)

# Title
st.title("🌐 Professional Language Translator")
st.markdown("Translate text, detect languages, and convert between text and speech with ease.")

# Initialize Translator
translator = Translator()

# Sidebar for Features
st.sidebar.title("Features")
feature = st.sidebar.radio(
    "Choose a feature",
    ["Text Translation", "Speech-to-Text", "Text-to-Speech", "Translation History"]
)

# Initialize Session State
if "history" not in st.session_state:
    st.session_state.history = []

# Function to Get Language Code
def get_language_code(language):
    return list(LANGUAGES.keys())[list(LANGUAGES.values()).index(language.lower())]

# Text Translation
if feature == "Text Translation":
    st.header("Text Translation")
    
    text_to_translate = st.text_area("Enter text to translate:", placeholder="Type something here...")
    
    source_language = st.selectbox(
        "Select source language:",
        ["Auto-detect"] + [lang.capitalize() for lang in LANGUAGES.values()]
    )
    
    target_language = st.selectbox(
        "Select target language:",
        [lang.capitalize() for lang in LANGUAGES.values()]
    )
    
    if st.button("Translate"):
        if text_to_translate.strip():
            try:
                # Determine source language code
                src_code = (
                    "auto"
                    if source_language == "Auto-detect"
                    else get_language_code(source_language)
                )
                dest_code = get_language_code(target_language)
                
                # Perform translation
                translation = translator.translate(text_to_translate, src=src_code, dest=dest_code)
                st.success(f"**Translated Text ({target_language}):**")
                st.write(translation.text)
                
                # Add to history
                st.session_state.history.append({
                    "source_text": text_to_translate,
                    "translated_text": translation.text,
                    "source_lang": source_language,
                    "target_lang": target_language
                })
            except Exception as e:
                st.error(f"Error: {e}")
        else:
            st.warning("Please enter text to translate!")

# Speech-to-Text
elif feature == "Speech-to-Text":
    st.header("Speech-to-Text")
    st.markdown("Upload an audio file, and we will convert it to text.")
    
    uploaded_file = st.file_uploader("Upload audio file (WAV format):", type=["wav"])
    
    if uploaded_file is not None:
        recognizer = sr.Recognizer()
        with sr.AudioFile(uploaded_file) as source:
            audio = recognizer.record(source)
        
        try:
            text = recognizer.recognize_google(audio)
            st.success("Converted Text:")
            st.write(text)
        except Exception as e:
            st.error(f"Error in speech recognition: {e}")

# Text-to-Speech
elif feature == "Text-to-Speech":
    st.header("Text-to-Speech")
    
    text_to_convert = st.text_area("Enter text to convert to speech:")
    language = st.selectbox("Select language:", [lang.capitalize() for lang in LANGUAGES.values()])
    
    if st.button("Convert to Speech"):
        if text_to_convert.strip():
            try:
                # Initialize the pyttsx3 engine
                engine = pyttsx3.init()

                # Set the properties for speed
                rate = engine.getProperty('rate')  # Get the current speech rate
                engine.setProperty('rate', rate )  # Multiply the rate by 5 for faster speech
                
                # Save the speech to a file
                audio_file = "output.mp3"
                engine.save_to_file(text_to_convert, audio_file)
                engine.runAndWait()  # Run the speech engine

                # Play Audio
                with open(audio_file, "rb") as file:
                    audio_bytes = file.read()
                    b64 = base64.b64encode(audio_bytes).decode()
                    href = f'<a href="data:audio/mp3;base64,{b64}" download="translated_audio.mp3">Download Audio</a>'
                    st.markdown(href, unsafe_allow_html=True)
                    st.audio(audio_bytes, format="audio/mp3")
                
                # Clean up
                os.remove(audio_file)
            except Exception as e:
                st.error(f"Error in text-to-speech conversion: {e}")
        else:
            st.warning("Please enter text to convert!")


# Translation History
elif feature == "Translation History":
    st.header("Translation History")
    
    if st.session_state.history:
        for i, entry in enumerate(st.session_state.history):
            st.write(f"**{i+1}. Source ({entry['source_lang']}):** {entry['source_text']}")
            st.write(f"**Translated ({entry['target_lang']}):** {entry['translated_text']}")
            st.markdown("---")
        
        if st.button("Clear History"):
            st.session_state.history = []
            st.success("Translation history cleared!")
    else:
        st.info("No translations yet!")

#Footer
st.markdown(
        """
        <style>
        .bottom-right {
            position: fixed;
            bottom: 10px;
            right: 15px;
            font-size: 0.9em;
            color: gray;
        }
        </style>
        <div class="bottom-right">
            Made with ⚡ at 'The Hackers Playbook' ©. All rights reserved.
        </div>
        """,
        unsafe_allow_html=True
                )



---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>
